# Advent of code 2021

## imports

In [1]:
import re
from itertools import combinations, chain
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Day 1

In [2]:
test_data = """199
200
208
210
200
207
240
269
260
263"""
test_data

'199\n200\n208\n210\n200\n207\n240\n269\n260\n263'

In [3]:
with open('./data_2021/d1p1.txt', 'r') as f:
    data = f.readlines()

In [4]:
data[:4]

['143\n', '147\n', '150\n', '166\n']

In [5]:
def clean_data(data):
    """Remove newlines and save as list of ints"""
    try:
        return np.array([int(d) for d in data.split('\n')])
    except AttributeError:
        return np.array([int(d) for d in data])
            
test_data = clean_data(test_data)
data = clean_data(data)

### Part 1

In [6]:
def count_increases(data):
    hi, lo = data[:-1], data[1:]
    deltas = lo - hi
    tot = len(deltas)
    for d in deltas:
        if d <= 0:
            tot -= 1
    return tot

count_increases(data) # 1532 correct for data, 7 for test

1532

### Part 2

In [7]:
from numpy.lib.stride_tricks import sliding_window_view

In [8]:
def count_sliding(data):
    slide = sliding_window_view(data, 3).sum(axis=1)
    wrongs = [1570, 1616]
    assert count_increases(slide) not in wrongs
    return count_increases(slide)

count_sliding(data) # 1571 correct

1571

## Day 2

### Part 1

In [9]:
with open('./data_2021/d2p1_test') as f:
    test = f.readlines()
with open('./data_2021/d2p1') as f:
    data = f.readlines()
    
start_hor, start_depth = 0, 0   
test, data[:5]

(['forward 5\n',
  'down 5\n',
  'forward 8\n',
  'up 3\n',
  'down 8\n',
  'forward 2\n'],
 ['forward 6\n', 'forward 9\n', 'down 9\n', 'down 7\n', 'forward 8\n'])

In [10]:
def get_pos(data, start_x, start_z):
    for line in data:
        val = int(line.split()[-1])
        if 'f' in line.lower():
            start_x += val
        elif 'd' in line.lower():
            start_z += val
        else:
            start_z -= val
    return start_x * start_z
    
assert get_pos(test, start_hor, start_depth) == 150
get_pos(data, start_hor, start_depth) # 2027977 correct

2027977

### Part 2

In [11]:
def get_course(data, start_x=0, start_z=0, start_aim=0):
    for line in data:
        move, val_str = line.split()
        move = move.lower()
        val = int(val_str)
        if move == 'down':
            start_aim += val
        elif move == 'up':
            start_aim -= val
        else:
            start_x += val
            start_z += start_aim * val
    
    return start_x * start_z

assert get_course(test) == 900
get_course(data) # 1903644897 correct

1903644897

## Day 3

### Part 1

In [12]:
with open('./data_2021/d3p1_test') as f:
    test = [x.strip() for x in f.readlines()]
with open('./data_2021/d3p1') as f:
    data = [x.strip() for x in f.readlines()]
print(test[:2], data[:2])

['00100', '11110'] ['000001110001', '000001111101']


In [13]:
def make_array(data):
    data_shape = len(data[0]), len(data)
    matrix = np.zeros(data_shape, dtype='int8')
    for idx, line in enumerate(data):
        raw = ' '.join(line)
        vec = np.fromstring(raw, dtype=int, sep=' ')
        matrix[:, idx] = vec
    return matrix.T

make_array(test)

array([[0, 0, 1, 0, 0],
       [1, 1, 1, 1, 0],
       [1, 0, 1, 1, 0],
       [1, 0, 1, 1, 1],
       [1, 0, 1, 0, 1],
       [0, 1, 1, 1, 1],
       [0, 0, 1, 1, 1],
       [1, 1, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0]], dtype=int8)

In [14]:
def power_consumption(array):
    gamma, epsilon = '', ''
    for i in range(array.shape[1]):
        gamma   += str(np.bincount(array[:,i]).argmax())
        epsilon += str(np.bincount(array[:,i]).argmin())
    return int(gamma, 2) * int(epsilon, 2), gamma, epsilon
    
power_consumption(make_array(data))[0] # 4118544 correct

4118544

### Part 2

In [100]:
def ox_gen_rate(array):
    _, gamma, epsilon = power_consumption(array)
    for idx, g in enumerate(gamma):
        arr = drop_rows(array, idx, g)
        
    return arr

def drop_rows(array, col_idx, value):
    mask = []
    for row_idx, row in enumerate(array):
        if row[col_idx] != value:
            mask += [False]
        else:
            mask += [True]
    mask = np.array(mask)
            
    return array[mask]

print(drop_rows(make_array(test), 0, 1))
print('=' * 25)
ox_gen_rate(make_array(test))

[[1 1 1 1 0]
 [1 0 1 1 0]
 [1 0 1 1 1]
 [1 0 1 0 1]
 [1 1 1 0 0]
 [1 0 0 0 0]
 [1 1 0 0 1]]


array([], shape=(0, 5), dtype=int8)

## D3P2 has me beat, recursion again.